In [14]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

In [15]:
load_dotenv()
api_key = os.environ.get("api_key")

In [18]:
# Variables: Note: these are case-sensitive
# from_symbol = 'USD'
to_symbol = 'RUB'

In [31]:
base_url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol={to_symbol}&outputsize=full&apikey={api_key}'
r = requests.get(base_url)
response_data = r.json()
response_data

{'Meta Data': {'1. Information': 'Forex Daily Prices (open, high, low, close)',
  '2. From Symbol': 'USD',
  '3. To Symbol': 'RUB',
  '4. Output Size': 'Full size',
  '5. Last Refreshed': '2022-09-29 22:10:00',
  '6. Time Zone': 'UTC'},
 'Time Series FX (Daily)': {'2022-09-30': {'1. open': '56.75000',
   '2. high': '56.75000',
   '3. low': '56.75000',
   '4. close': '56.75000'},
  '2022-09-29': {'1. open': '57.88420',
   '2. high': '58.52500',
   '3. low': '56.00000',
   '4. close': '56.00000'},
  '2022-09-28': {'1. open': '58.89330',
   '2. high': '59.19200',
   '3. low': '56.87300',
   '4. close': '57.88160'},
  '2022-09-27': {'1. open': '58.48880',
   '2. high': '59.40000',
   '3. low': '57.27500',
   '4. close': '58.89590'},
  '2022-09-26': {'1. open': '57.05000',
   '2. high': '58.86190',
   '3. low': '56.00000',
   '4. close': '58.48980'},
  '2022-09-23': {'1. open': '59.87250',
   '2. high': '60.23500',
   '3. low': '55.67500',
   '4. close': '56.87000'},
  '2022-09-22': {'1. op

In [34]:
def extract_fx_rates(
    from_symbol:str,
    to_symbol:str,
    api_key:str
)->pd.DataFrame:
    """
    Extracts daily exchange rates.
    - `from_symbol`: The currency symbol from which the exchange occurs.
    - `to_symbol`: The currency symbol into which the exchange occurs.
    
    """
    base_url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=USD&to_symbol={to_symbol}&outputsize=full&apikey={api_key}'
    r = requests.get(base_url)
    if r.status_code == 200:
        response_data = r.json()
        df = pd.DataFrame(response_data['Time Series FX (Daily)']).transpose().reset_index()
        df = df.rename(columns={ df.columns[0]: "Date" })
        # df['from'] = f'{from_symbol}' # Won't use in case we do only USD in the FROM column
        df['to'] = f'{to_symbol}'
        return df

In [35]:
df = extract_fx_rates(from_symbol=from_symbol, to_symbol=to_symbol, api_key=api_key)
df

,Date,1. open,2. high,3. low,4. close,to
0,2022-09-30,56.75000,56.75000,56.75000,56.75000,RUB
1,2022-09-29,57.88420,58.52500,56.00000,56.00000,RUB
2,2022-09-28,58.89330,59.19200,56.87300,57.88160,RUB
3,2022-09-27,58.48880,59.40000,57.27500,58.89590,RUB
4,2022-09-26,57.05000,58.86190,56.00000,58.48980,RUB
...,...,...,...,...,...,...
2045,2014-11-28,48.63850,50.68550,48.61600,50.47380,RUB
2046,2014-11-27,47.39500,49.27400,46.68890,49.09690,RUB
2047,2014-11-26,46.30000,47.42130,46.01750,47.39360,RUB
2048,2014-11-25,44.94000,46.34500,44.59670,46.31390,RUB


In [36]:
def extract():
    df_currencies = pd.read_csv("data/main_currencies.csv")
    df_concat = pd.DataFrame()
    for currency_name in df_currencies["currency code"]:
        df_extracted = extract_fx_rates(from_symbol=from_symbol, to_symbol=currency_name, api_key=api_key)
        df_concat = pd.concat([df_concat, df_extracted])
    return df_concat.reset_index().drop(labels=["index"], axis=1)

In [21]:
# Trying to figure out: how to use parameter as df name to create df per exchange currency?
# (Unrecommended?) way of doing this: https://stackoverflow.com/questions/67117157/is-there-a-way-to-made-the-name-of-a-dataframe-a-variable-that-is-defined-by-the
# def extract():
#     df_currencies = pd.read_csv("data/main_currencies.csv")
#     df_concat = pd.DataFrame()
#     for val in df_currencies["currency code"]:
#         globals()["table_"f"{val}"] = extract_fx_rates(from_symbol=from_symbol, to_symbol=val, api_key=api_key)
#         return globals()["table_"f"{val}"].reset_index().drop(labels=["index"], axis=1)

# df_currencies = pd.read_csv("data/main_currencies.csv")
# for currency_name, val in enumerate((df_currencies["currency code"])):
#     print(f'currency_name_{val}')

In [50]:
extract()

,Date,1. open,2. high,3. low,4. close,to
0,2022-09-30,1.01690,1.01840,1.01680,1.01780,EUR
1,2022-09-29,1.02690,1.03730,1.01780,1.01830,EUR
2,2022-09-28,1.04150,1.04810,1.02530,1.02700,EUR
3,2022-09-27,1.04010,1.04450,1.03420,1.04190,EUR
4,2022-09-26,1.03260,1.04660,1.02980,1.04050,EUR
...,...,...,...,...,...,...
13187,2014-11-28,1.17290,1.17820,1.17060,1.17500,AUD
13188,2014-11-27,1.17050,1.17460,1.16040,1.17320,AUD
13189,2014-11-26,1.17180,1.17900,1.16700,1.17090,AUD
13190,2014-11-25,1.16060,1.17430,1.16020,1.17180,AUD


In [25]:
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, Float # https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm
from sqlalchemy.engine import URL
from sqlalchemy.dialects import postgresql
from sqlalchemy.schema import CreateTable 

In [26]:
db_user = os.environ.get("db_user")
db_password = os.environ.get("db_password")
db_server_name = os.environ.get("db_server_name")
db_database_name = os.environ.get("db_database_name")

# create connection to database 
connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5432,
    database = db_database_name, 
)
engine = create_engine(connection_url)


In [52]:
def load(
    df:pd.DataFrame,
    load_method:str="overwrite",
    target_database_engine=engine,
    target_table_name:str=None
    )->None:
    """
    Load dataframe to either a file or a database. 
    - df: pandas dataframe to load.  
    - load_target: choose either `file` or `database`.
    - load_method: choose either `overwrite` or `upsert`. defaults to `overwrite`. 
    - target_file_directory: directory where the file will be written to in parquet format.
    - target_file_name: name of the target file e.g. stock.parquet. 
    - target_database_engine: SQLAlchemy engine for the target database. 
    - target_table_name: name of the SQL table to create and/or upsert data to. 
    """
    if load_method.lower() == "overwrite": 
        df.to_sql(target_table_name, target_database_engine)
    elif load_method.lower() == "upsert":
        meta = MetaData()
        fx_table = Table(
            target_table_name, meta, 
            Column("date", String, primary_key=True),
            Column("open_value", Integer),
            Column("high_value", String),
            Column("low_value", Float),
            Column("close_value", Integer),
            # Column("from", String, primary_key=True),  # Won't use in case we do only USD in the FROM column
            Column("to", String, primary_key=True)
        )
        meta.create_all(target_database_engine) # creates table if it does not exist 
        insert_statement = postgresql.insert(fx_table).values(df.to_dict(orient='records'))
        upsert_statement = insert_statement.on_conflict_do_update(
            index_elements=['date', 'to'],
            set_={c.key: c for c in insert_statement.excluded if c.key not in ['date', 'to']})
        target_database_engine.execute(upsert_statement)

In [ ]:
load(df=extract(), target_table_name='usd_exchange_rates')